In [ ]:
import csv
import collections

def _get_example_objects(annon_filepath):
    """Function to get all the objects from the annotation CSV file."""
    with tf.io.gfile.GFile(annon_filepath, "r") as f:
        
        root = xml.etree.ElementTree.parse(f).getroot()
        
        size = root.find("size")
        width = float(size.find("width").text)
        height = float(size.find("height").text)

        for obj in root.findall("object"):
            # Get object's label name.
            label = obj.find("name").text.lower()
            # Get objects' pose name.
            pose = obj.find("pose").text.lower()
            is_truncated = (obj.find("truncated").text == "1")
            is_difficult = (obj.find("difficult").text == "1")
            bndbox = obj.find("bndbox")
            xmax = float(bndbox.find("xmax").text)
            xmin = float(bndbox.find("xmin").text)
            ymax = float(bndbox.find("ymax").text)
            ymin = float(bndbox.find("ymin").text)
            yield {
              "label": label,
              "pose": pose,
              "bbox": tfds.features.BBox(
                  ymin / height, xmin / width, ymax / height, xmax / width),
            }
    # pytype: enable=attribute-error


In [65]:
import logging
import collections
import csv

_Bbox = collections.namedtuple('Bbox', [
    'label', 'source', 'bbox',])


def _load_bboxes(csv_path, csv_positions, prefix):
    """Returns bounded boxes listed within given CSV file."""
    logging.info('Loading CSVs %s from positions %s with prefix %s',
               csv_path, csv_positions, prefix)
    boxes = collections.defaultdict(list)
    with tf.io.gfile.GFile(csv_path) as csv_f:
        if csv_positions[0] > 0:
            csv_f.seek(csv_positions[0])
        else:
            csv_f.readline()  # Drop headers
        #csv_f.readline()
        reader = csv.reader(csv_f)
    i = 0
    for (filename, width, height, classes, xmin, ymin, xmax, ymax) in reader:
        if filename == 'fileName':
            break
#         csv_positions[0] = csv_f.tell()
#         image_id = int(image_id, 16)
#         del confidence  # always 1 in bounding boxes.
        print(filename)
        current_row = _Bbox(
          classes, filename, tfds.features.BBox(
              float(ymin), float(xmin), float(ymax), float(xmax)))
        boxes[i].append(current_row)
        i = i + 1
    return dict(boxes)

In [67]:
_load_bboxes('./dataset/excel/comptech_train.csv', [22:55], 'fileName')

SyntaxError: invalid syntax (<ipython-input-67-9cf02bfd9304>, line 1)

In [69]:
import numpy
numpy.random.choice(numpy.arange(1, 7), p=[0.1, 0.05, 0.05, 0.2, 0.4, 0.2])

4

In [21]:
"""Texture tiles from colorectal cancer histology."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import tensorflow.compat.v2 as tf

from PIL import Image
import tensorflow_datasets.public_api as tfds


_TILES_DL_URL = "dataset/slice_data_on_folders/both.zip"
_TILES_SUBDIR = "dataset/slice_data_on_folders/both"

_CLASS_NAME = (
    "wings",
)

_TILES_SIZE = 1920
_TILES_SIZE_WIDTH = 1080

# Load image in numpy array
def _load_tif(path):
    with tf.io.gfile.GFile(path, "rb") as fp:
        image = tfds.core.lazy_imports.PIL_Image.open(fp)
    return np.array(image)


class WingsDataset(tfds.core.GeneratorBasedBuilder):
    """Detection Wings On Images."""
    VERSION = tfds.core.Version(
      "2.0.0", "New split API (https://tensorflow.org/datasets/splits)")
    SUPPORTED_VERSIONS = [
      tfds.core.Version("0.0.1",
                        experiments={tfds.core.Experiment.S3: False}),
    ]

    def _info(self):
        return tfds.core.DatasetInfo(
            builder=self,
            description=(
                "Classification of textures in colorectal cancer histology. "
                "Each example is a 1920 x 1080 x 3 RGB image of one of 1 classes."),
            features=tfds.features.FeaturesDict({
                "image": tfds.features.Image(shape=(_TILES_SIZE, _TILES_SIZE_WIDTH) + (3,)),
                "filename": tfds.features.Text(),
                "objects": tfds.features.Sequence({
                    "label": tfds.features.ClassLabel(names=_CLASS_NAME),
                    "bbox": tfds.features.BBoxFeature()
                }),
            }),
            supervised_keys=("image", "label"),
        )
    
    def _split_generators(self):
        pass
    
    def _generate_examples(self, root_dir):
        print('a')
        root_dir = os.path.join(root_dir, _TILES_SUBDIR)
        print(root_dir)
        for i, class_name in enumerate(_CLASS_NAME):
            fns = tf.io.gfile.listdir(root_dir)
            for fn in sorted(fns):
                image = _load_tif(os.path.join(root_dir, fn))
                record = {
                    "image": image,
                #                 "label": class_name,
                    "filename": fn,

                }
                yield "%s/%s" % (class_name, fn), record

    def _generate_example(self, data_path, image_id, load_annotations):
        image_filepath = os.path.join(
            data_path,
            os.path.normpath("VOCdevkit/VOC{}/JPEGImages/{}.jpg".format(
                self.builder_config.year, image_id)))

        annon_filepath = os.path.join(
            data_path,
            os.path.normpath("VOCdevkit/VOC{}/Annotations/{}.xml".format(
                self.builder_config.year, image_id)))
        if load_annotations:
            objects = list(_get_example_objects(annon_filepath))
            # Use set() to remove duplicates
            labels = sorted(set(obj["label"] for obj in objects))
        else:  # The test set of VOC2012 does not contain annotations
            objects = []
            labels = []
            
        return {
            "image": image_filepath,
            "image/filename": image_id + ".jpg",
            "objects": objects,
            "labels": labels,
            "labels_no_difficult": labels_no_difficult,
        }

In [22]:
dataset = WingsDataset()

In [23]:
dataset.info

tfds.core.DatasetInfo(
    name='wings_dataset',
    version=2.0.0,
    description='Classification of textures in colorectal cancer histology. Each example is a 150 x 150 x 3 RGB image of one of 8 classes.',
    homepage='https://www.tensorflow.org/datasets/catalog/wings_dataset',
    features=FeaturesDict({
        'filename': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(1920, 1080, 3), dtype=tf.uint8),
        'objects': Sequence({
            'bbox': BBoxFeature(shape=(4,), dtype=tf.float32),
            'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=1),
        }),
    }),
    total_num_examples=0,
    splits={
    },
    supervised_keys=('image', 'label'),
    citation="""""",
    redistribution_info=,
)

In [14]:
a = dataset._generate_examples('./')
for i in a:
    print(i)

a
./dataset/slice_data_on_folders/both
source1_001.bmp
('wings/source1_001.bmp', {'image': array([[[145, 145, 145],
        [145, 145, 145],
        [144, 144, 144],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[147, 147, 147],
        [147, 147, 147],
        [146, 146, 146],
        ...,
        [121, 121, 121],
        [121, 121, 121],
        [121, 121, 121]],

       [[149, 149, 149],
        [148, 148, 148],
        [147, 147, 147],
        ...,
        [124, 124, 124],
        [124, 124, 124],
        [124, 124, 124]],

       ...,

       [[134, 134, 134],
        [134, 134, 134],
        [135, 135, 135],
        ...,
        [182, 182, 182],
        [181, 181, 181],
        [180, 180, 180]],

       [[135, 135, 135],
        [135, 135, 135],
        [136, 136, 136],
        ...,
        [180, 180, 180],
        [181, 181, 181],
        [181, 181, 181]],

       [[135, 135, 135],
        [135, 135, 135],
        [136, 136, 13

('wings/source1_055.bmp', {'image': array([[[145, 145, 145],
        [143, 143, 143],
        [141, 141, 141],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [125, 125, 125]],

       [[143, 143, 143],
        [143, 143, 143],
        [143, 143, 143],
        ...,
        [131, 131, 131],
        [128, 128, 128],
        [122, 122, 122]],

       [[144, 144, 144],
        [147, 147, 147],
        [149, 149, 149],
        ...,
        [121, 121, 121],
        [118, 118, 118],
        [113, 113, 113]],

       ...,

       [[135, 135, 135],
        [135, 135, 135],
        [135, 135, 135],
        ...,
        [183, 183, 183],
        [186, 186, 186],
        [178, 178, 178]],

       [[135, 135, 135],
        [135, 135, 135],
        [135, 135, 135],
        ...,
        [178, 178, 178],
        [191, 191, 191],
        [180, 180, 180]],

       [[136, 136, 136],
        [136, 136, 136],
        [136, 136, 136],
        ...,
        [177, 177, 177],
        [188,

('wings/source1_110.bmp', {'image': array([[[151, 151, 151],
        [149, 149, 149],
        [148, 148, 148],
        ...,
        [136, 136, 136],
        [133, 133, 133],
        [130, 130, 130]],

       [[148, 148, 148],
        [148, 148, 148],
        [148, 148, 148],
        ...,
        [131, 131, 131],
        [126, 126, 126],
        [121, 121, 121]],

       [[145, 145, 145],
        [148, 148, 148],
        [149, 149, 149],
        ...,
        [129, 129, 129],
        [124, 124, 124],
        [119, 119, 119]],

       ...,

       [[140, 140, 140],
        [140, 140, 140],
        [141, 141, 141],
        ...,
        [180, 180, 180],
        [182, 182, 182],
        [180, 180, 180]],

       [[138, 138, 138],
        [136, 136, 136],
        [137, 137, 137],
        ...,
        [182, 182, 182],
        [181, 181, 181],
        [182, 182, 182]],

       [[140, 140, 140],
        [138, 138, 138],
        [139, 139, 139],
        ...,
        [182, 182, 182],
        [182,

source1_166.bmp
('wings/source1_166.bmp', {'image': array([[[120, 120, 120],
        [131, 131, 131],
        [146, 146, 146],
        ...,
        [133, 133, 133],
        [125, 125, 125],
        [121, 121, 121]],

       [[136, 136, 136],
        [148, 148, 148],
        [159, 159, 159],
        ...,
        [127, 127, 127],
        [117, 117, 117],
        [115, 115, 115]],

       [[153, 153, 153],
        [153, 153, 153],
        [150, 150, 150],
        ...,
        [132, 132, 132],
        [121, 121, 121],
        [119, 119, 119]],

       ...,

       [[136, 136, 136],
        [137, 137, 137],
        [138, 138, 138],
        ...,
        [179, 179, 179],
        [184, 184, 184],
        [180, 180, 180]],

       [[137, 137, 137],
        [136, 136, 136],
        [135, 135, 135],
        ...,
        [181, 181, 181],
        [186, 186, 186],
        [183, 183, 183]],

       [[142, 142, 142],
        [141, 141, 141],
        [140, 140, 140],
        ...,
        [180, 180, 180

('wings/source1_212.bmp', {'image': array([[[121, 121, 121],
        [127, 127, 127],
        [137, 137, 137],
        ...,
        [135, 135, 135],
        [129, 129, 129],
        [124, 124, 124]],

       [[134, 134, 134],
        [149, 149, 149],
        [157, 157, 157],
        ...,
        [128, 128, 128],
        [124, 124, 124],
        [122, 122, 122]],

       [[145, 145, 145],
        [155, 155, 155],
        [152, 152, 152],
        ...,
        [123, 123, 123],
        [121, 121, 121],
        [120, 120, 120]],

       ...,

       [[138, 138, 138],
        [138, 138, 138],
        [139, 139, 139],
        ...,
        [179, 179, 179],
        [187, 187, 187],
        [180, 180, 180]],

       [[140, 140, 140],
        [138, 138, 138],
        [138, 138, 138],
        ...,
        [183, 183, 183],
        [188, 188, 188],
        [175, 175, 175]],

       [[137, 137, 137],
        [136, 136, 136],
        [136, 136, 136],
        ...,
        [182, 182, 182],
        [189,

...

('wings/source3_093.bmp', {'image': array([[[135, 135, 135],
        [138, 138, 138],
        [138, 138, 138],
        ...,
        [114, 114, 114],
        [134, 134, 134],
        [164, 164, 164]],

       [[133, 133, 133],
        [134, 134, 134],
        [133, 133, 133],
        ...,
        [118, 118, 118],
        [113, 113, 113],
        [131, 131, 131]],

       [[127, 127, 127],
        [129, 129, 129],
        [129, 129, 129],
        ...,
        [ 89,  89,  89],
        [ 71,  71,  71],
        [ 73,  73,  73]],

       ...,

       [[111, 111, 111],
        [110, 110, 110],
        [117, 117, 117],
        ...,
        [176, 176, 176],
        [178, 178, 178],
        [179, 179, 179]],

       [[115, 115, 115],
        [114, 114, 114],
        [115, 115, 115],
        ...,
        [177, 177, 177],
        [177, 177, 177],
        [178, 178, 178]],

       [[116, 116, 116],
        [116, 116, 116],
        [114, 114, 114],
        ...,
        [178, 178, 178],
        [177,

('wings/source3_142.bmp', {'image': array([[[131, 131, 131],
        [133, 133, 133],
        [139, 139, 139],
        ...,
        [108, 108, 108],
        [143, 143, 143],
        [168, 168, 168]],

       [[135, 135, 135],
        [135, 135, 135],
        [139, 139, 139],
        ...,
        [ 86,  86,  86],
        [ 94,  94,  94],
        [107, 107, 107]],

       [[138, 138, 138],
        [135, 135, 135],
        [135, 135, 135],
        ...,
        [ 77,  77,  77],
        [ 70,  70,  70],
        [ 71,  71,  71]],

       ...,

       [[123, 123, 123],
        [123, 123, 123],
        [123, 123, 123],
        ...,
        [181, 181, 181],
        [178, 178, 178],
        [175, 175, 175]],

       [[123, 123, 123],
        [123, 123, 123],
        [123, 123, 123],
        ...,
        [182, 182, 182],
        [180, 180, 180],
        [178, 178, 178]],

       [[123, 123, 123],
        [123, 123, 123],
        [123, 123, 123],
        ...,
        [182, 182, 182],
        [181,

('wings/source3_197.bmp', {'image': array([[[135, 135, 135],
        [141, 141, 141],
        [147, 147, 147],
        ...,
        [119, 119, 119],
        [147, 147, 147],
        [164, 164, 164]],

       [[142, 142, 142],
        [146, 146, 146],
        [149, 149, 149],
        ...,
        [145, 145, 145],
        [101, 101, 101],
        [ 77,  77,  77]],

       [[144, 144, 144],
        [143, 143, 143],
        [141, 141, 141],
        ...,
        [101, 101, 101],
        [ 98,  98,  98],
        [ 98,  98,  98]],

       ...,

       [[109, 109, 109],
        [111, 111, 111],
        [110, 110, 110],
        ...,
        [176, 176, 176],
        [181, 181, 181],
        [181, 181, 181]],

       [[107, 107, 107],
        [110, 110, 110],
        [110, 110, 110],
        ...,
        [165, 165, 165],
        [169, 169, 169],
        [170, 170, 170]],

       [[106, 106, 106],
        [109, 109, 109],
        [109, 109, 109],
        ...,
        [174, 174, 174],
        [175,

('wings/source3_243.bmp', {'image': array([[[129, 129, 129],
        [131, 131, 131],
        [140, 140, 140],
        ...,
        [135, 135, 135],
        [139, 139, 139],
        [134, 134, 134]],

       [[138, 138, 138],
        [138, 138, 138],
        [143, 143, 143],
        ...,
        [119, 119, 119],
        [117, 117, 117],
        [114, 114, 114]],

       [[135, 135, 135],
        [136, 136, 136],
        [141, 141, 141],
        ...,
        [ 92,  92,  92],
        [ 91,  91,  91],
        [ 96,  96,  96]],

       ...,

       [[112, 112, 112],
        [113, 113, 113],
        [111, 111, 111],
        ...,
        [184, 184, 184],
        [180, 180, 180],
        [180, 180, 180]],

       [[108, 108, 108],
        [111, 111, 111],
        [113, 113, 113],
        ...,
        [176, 176, 176],
        [172, 172, 172],
        [172, 172, 172]],

       [[106, 106, 106],
        [110, 110, 110],
        [114, 114, 114],
        ...,
        [184, 184, 184],
        [178,

('wings/source3_286.bmp', {'image': array([[[131, 131, 131],
        [136, 136, 136],
        [143, 143, 143],
        ...,
        [126, 126, 126],
        [128, 128, 128],
        [130, 130, 130]],

       [[144, 144, 144],
        [145, 145, 145],
        [143, 143, 143],
        ...,
        [125, 125, 125],
        [131, 131, 131],
        [124, 124, 124]],

       [[142, 142, 142],
        [139, 139, 139],
        [133, 133, 133],
        ...,
        [ 91,  91,  91],
        [ 90,  90,  90],
        [ 85,  85,  85]],

       ...,

       [[110, 110, 110],
        [111, 111, 111],
        [111, 111, 111],
        ...,
        [177, 177, 177],
        [177, 177, 177],
        [179, 179, 179]],

       [[110, 110, 110],
        [111, 111, 111],
        [111, 111, 111],
        ...,
        [177, 177, 177],
        [176, 176, 176],
        [177, 177, 177]],

       [[110, 110, 110],
        [111, 111, 111],
        [111, 111, 111],
        ...,
        [178, 178, 178],
        [178,

In [10]:
iterator = a.make_one_shot_iterator()

AttributeError: 'generator' object has no attribute 'make_one_shot_iterator'

In [ ]:
q = a._generate_examples('./')

In [66]:
for i in q:
    print(i)